## Using Abeta pattern

In [2]:
import nibabel as nib
from nilearn.image import resample_to_img, smooth_img, resample_img, new_img_like
import numpy as np
from nilearn.masking import apply_mask, unmask
from nilearn.datasets import load_mni152_brain_mask, load_mni152_brain_mask

mni152 = nib.load('./example_img.nii')
brain_wo_voi_mask = load_mni152_brain_mask()
brain_2mm_mask = resample_to_img(brain_wo_voi_mask, mni152, interpolation='nearest')

def preprocess_img(pet_img, downsample=3):
    pet_img = smooth_img(pet_img, fwhm=8)
    data = pet_img.get_fdata()
    data[np.isnan(data)] = 0
    pet_img = nib.nifti1.Nifti1Image(data, pet_img.affine, pet_img.header)
    downsample_affine = pet_img.affine.copy()
    downsample_affine[:3, :3] *= downsample
    pet_img = resample_img(pet_img, target_affine=downsample_affine)
    return pet_img

def scale_intensity(X:np.array):
    return (X - X.min(axis=1).reshape((-1, 1))) / (X.max(axis=1) - X.min(axis=1)).reshape(-1,1)

def flip_image(img):
    data = img.get_fdata()
    data = np.flip(data, 0)
    new_image = new_img_like(img, data)
    return new_image

def whole_brain_to_half(img, mask):
    return [apply_mask(flip_image(img), mask), apply_mask(img, mask)]

def half_to_whole_brain(half_imgs, mask):
    return unmask(half_imgs[0], mask)+unmask(half_imgs[1], mask)

In [ ]:
# if you wish to build a logistic regression model

# from sklearn.linear_model import LogisticRegression
# import numpy as np
# from sklearn.metrics import roc_auc_score, roc_curve, auc

# X = []
# for img in images:
#     img = preprocess_img(img, downsample=3)
#     X.extend(whole_brain_to_half(img, brain_2mm_mask))
# X = scale_intensity(np.array(X))
# y = ...

# lr = LogisticRegression(max_iter=50000, class_weight='balanced')
# lr.fit(X, y)
# print(roc_auc_score(y_test, lr.decision_function(X_test)))
# print(classification_report(y_test, lr.predict(X_test)))

In [4]:
# to use the trained model
# please note that pet images should be normalized first!

from pathlib import Path

import pickle
with open('./LCAD_LCCN_lr_model.pkl', 'rb') as f:
    lr = pickle.load(f)


downsample_ratio = 3
example_img = nib.load('./example_img.nii')
example_img = preprocess_img(example_img, downsample_ratio)
resampled_mask = resample_to_img(brain_2mm_mask, example_img, interpolation='nearest')
example_img = apply_mask(example_img, resampled_mask)
X = scale_intensity(np.array([example_img]))
lr.decision_function(X), lr.predict(X)

(array([6.10816421]), array([1]))

## Using DCCC to calculate Centiloid

In [6]:
from affine_voxelmorph import AffineVoxelMorph, warp
from rigid import RegressorModel, dl_rigid_transform
import nibabel as nib
import torch
from pathlib import Path
import pickle

inshape = (1, 64, 64, 64)
channels = (64, 64, 64, 128, 256)
strides = (2, 2, 2, 2, 2)
device = "cuda" if torch.cuda.is_available() else "cpu"
rigid_model = RegressorModel(inshape, channels, strides)
rigid_model.load_state_dict(torch.load('./Rigid.pth'))
_ = rigid_model.to(device).eval()
affine_vm_model = AffineVoxelMorph()
affine_vm_model.load_state_dict(torch.load('./AffineVoxelMorph.pth'))
_ = affine_vm_model.to(device).eval()

C:\Users\85926\AppData\Local\Temp\ipykernel_40332\1661998562.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  rigid_model.load_state_dict(torch.load('./Rigid.pth'))
e:\p

In [14]:
voi_ctx = nib.load("./voi_ctx_2mm.nii")
wc_ref = nib.load("./voi_WhlCbl_2mm.nii")
def calc_suvr(img, voi, ref):
    img = resample_to_img(img, voi)
    voi = apply_mask(img, voi)
    ref = apply_mask(img, ref)
    return voi.mean() / ref.mean()

def get_centiloid(suvr, tracer):
    if tracer == 'av45':
        centiloid = suvr * 175.4 - 182.3
    elif tracer == 'pib':
        centiloid = suvr * 93.7 - 94.6
    elif tracer == 'fbb':
        centiloid = suvr * 153.4 - 154.9
    elif tracer == 'fmm':
        centiloid = suvr * 83.7 - 94.6
    else:
        raise NotImplementedError()
    return centiloid

rigid_img, *_ = dl_rigid_transform('./YC12_AV45.nii', rigid_model)
nib.save(rigid_img, './rigid.nii')
warped_img = warp('./rigid.nii', './template_pet.nii', affine_vm_model)
pred_suvr = calc_suvr(warped_img, voi_ctx, wc_ref)
get_centiloid(pred_suvr, 'av45')

e:\programs\Anaconda\envs\dlmed\lib\site-packages\monai\networks\blocks\warp.py:72: UserWarning: monai.networks.blocks.Warp: Using PyTorch native grid_sample.
  warnings.warn("monai.networks.blocks.Warp: Using PyTorch native grid_sample.")


4.469583344459522

In [17]:
rigid_img, *_ = dl_rigid_transform('./ED25_AV45.nii', rigid_model)
nib.save(rigid_img, './rigid.nii')
warped_img = warp('./rigid.nii', './template_pet.nii', affine_vm_model)
pred_suvr = calc_suvr(warped_img, voi_ctx, wc_ref)
get_centiloid(pred_suvr, 'av45')

71.77903392314911